In [1]:
# 加载包
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
import datetime

In [2]:
# 参数设置
## 增加重连次数
requests.adapters.DEFAULT_RETRIES = 5
base_url = 'http://www.trendiano.com/'
url_end0 = 'list.shtml'
url_end1 = 'list-'
url_end2 = '-30-listedDate%20desc-0-0-0-0-0.shtml'
url_end3 = '-30-catalog%20desc-0-0-0-0-0.shtml'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 UBrowser/6.1.2107.204 Safari/537.36',
          'Connection':'close'}

In [4]:
# 类目名称、URL、页码
cat_list = pd.read_table('./trendiano_cat_list.csv', sep = ',', encoding = 'gbk')
cat_list

,cat1_name,cat2_name,cat2_href,page_num
0,T恤/衬衫,无袖T恤,/T_shirts/Sleeveless_T_shirts/,1
1,T恤/衬衫,短袖T恤,/T_shirts/Short_sleeved_T_shirts/,6
2,T恤/衬衫,长袖T恤,/T_shirts/Long_sleeved_T_shirts/,1
3,T恤/衬衫,POLO,/T_shirts/POLO_shirts/,1
4,T恤/衬衫,长袖衬衫,/T_shirts/Long_sleeved_shirts/,3
5,T恤/衬衫,短袖衬衫,/T_shirts/Short_sleeved_shirts/,1
6,卫衣/毛织,卫衣,/Hoodies_Woolen/Hoodies/,5
7,卫衣/毛织,毛织,/Hoodies_Woolen/Woollen/,2
8,裤装,休闲裤-长裤,/Pants/Casual_pants/Long_casual_pants/,3
9,裤装,休闲裤-九分裤,/Pants/Casual_pants/Ankle_length_casual_pants/,5


In [5]:
# 爬取商品信息和图片
print('开始')
## 打开 csv 文件，encoding = 'utf-8-sig'，保证写入中文不乱码
csv_file = open('trendiano_goods_info.csv', 'w', newline = '', encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
## 命名字段
csv_writer.writerow(['cat1_name', 'cat2_name', 'goods_name', 'skc_id', 
                     'sale_price', 'origin_price','href', 'style', 
                     'material', 'length', 'stereotype', 'img_src'])
for row_num in range(cat_list.shape[0]):
    for page_num in range(cat_list.page_num[row_num]):
        # 二级类目商品列表页面中的商品主信息
        ## 页面
        s1 = requests.Session()
        s1.keep_alive = False
        url1 = base_url + cat_list.cat2_href[row_num] + url_end1 + str(page_num + 1) + url_end2
        page1 = s1.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        ## 商品主信息
        goods_infos = source1.find_all('div', class_ = 'chip')
        for goods_info in goods_infos:
            goods_name = goods_info.find('span', class_ = 'chip_name').string.replace('\n',"")
            skc_id = goods_info.find('a').get('data-sku').replace('\n',"")
            sale_price = goods_info.find('p', class_ = 'chip_price').span.string.replace('\n',"")
            try:
                origin_price = goods_info.find('p', class_ = 'chip_price').find('del').string
            except Exception as e:
                origin_price = sale_price
            href = goods_info.find('a').get('href')
            # 商品详情页面中的商品辅信息和图片
            ## 页面
            s2 = requests.session()
            s2.keep_alive = False
            url2 = base_url + href
            page2 = s2.get(url2, headers = header)
            source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
            ## 商品信息
            goods_info2 = source2.find('dl', class_ = 'pro_description').find_all('dl')
            style = goods_info2[1].dd.string.replace('\n',"")
            try:
                material = goods_info2[3].dd.string.replace('\n',"")
            except Exception as e:
                material = None
            try:
                length = goods_info2[4].dd.string.replace('\n',"")
            except Exception as e:
                length = None
            try:
                stereotype = goods_info2[5].dd.string.replace('\n',"")
            except Exception as e:
                stereotype = None  
            img_src = source2.find('div', class_ = 'product_img_show').find('img').get('src')
            ## 把商品信息写入到 csv 文件中
            csv_writer.writerow([cat_list.cat1_name[row_num], cat_list.cat2_name[row_num], 
                                 goods_name, skc_id, sale_price, origin_price, href,
                                 style, material, length, stereotype, img_src])
            '''
            ## 保存商品主图到本地文件夹
            img = requests.get(img_src, timeout = 3)
            img_name = "trendiano_goods_img/"+ skc_id + ".jpg"
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            ''' 
            '''
            ## 显示单个商品进度
            print(cat_list.cat1_name[row_num], cat_list.cat2_name[row_num], '第', page_num+1, '页')
            print(skc_id)
            print('------------')
            ''' 
        ## 显示进度
        print(cat_list.cat1_name[row_num], cat_list.cat2_name[row_num], '第', page_num+1, '页')
        print('############')
        ## 每翻一页，休息一下 
        time.sleep(10)
    ## 每爬取一个品类，休息一下
    time.sleep(30)
## 关闭csv文件
csv_file.close()

print('结束')

开始
T恤/衬衫 无袖T恤 第 1 页
############
T恤/衬衫 短袖T恤 第 1 页
############
T恤/衬衫 短袖T恤 第 2 页
############
T恤/衬衫 短袖T恤 第 3 页
############
T恤/衬衫 短袖T恤 第 4 页
############
T恤/衬衫 短袖T恤 第 5 页
############
T恤/衬衫 短袖T恤 第 6 页
############
T恤/衬衫 长袖T恤 第 1 页
############
T恤/衬衫 POLO 第 1 页
############
T恤/衬衫 长袖衬衫 第 1 页
############
T恤/衬衫 长袖衬衫 第 2 页
############
T恤/衬衫 长袖衬衫 第 3 页
############
T恤/衬衫 短袖衬衫 第 1 页
############
卫衣/毛织 卫衣 第 1 页
############
卫衣/毛织 卫衣 第 2 页
############
卫衣/毛织 卫衣 第 3 页
############
卫衣/毛织 卫衣 第 4 页
############
卫衣/毛织 卫衣 第 5 页
############
卫衣/毛织 毛织 第 1 页
############
卫衣/毛织 毛织 第 2 页
############
裤装 休闲裤-长裤 第 1 页
############
裤装 休闲裤-长裤 第 2 页
############
裤装 休闲裤-长裤 第 3 页
############
裤装 休闲裤-九分裤 第 1 页
############
裤装 休闲裤-九分裤 第 2 页
############
裤装 休闲裤-九分裤 第 3 页
############
裤装 休闲裤-九分裤 第 4 页
############
裤装 休闲裤-九分裤 第 5 页
############
裤装 休闲裤-短裤 第 1 页
############
裤装 休闲裤-短裤 第 2 页
############
裤装 牛仔裤 第 1 页
############
外套 风衣 第 1 页
############
外套 风衣 第 2 页
############
外套 风衣 第 3 页
############
外套 夹克 第 1 页
###

In [6]:
# ####################
# 爬取最近两个月上市的新品
# ####################

newin_url = ['New_in/New_in_11_22/','New_in/New_in_11_10/','New_in/New_in_10_25/', 'New_in/New_in_10_11/']
newin_date = ['2018-11-22', '2018-11-10', '2018-10-25','2018-10-11']
newin_page = [2, 3, 5, 4]
# 打开 csv 文件，encoding = 'utf-8-sig'，保证写入中文不乱码
csv_file = open('trendiano_goods_newin.csv', 'w', newline = '', encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
## 命名字段
csv_writer.writerow(['skc_id', 'newin_date'])
for row_num in range(len(newin_date)):
    for page_num in range(newin_page[row_num]):
        # 新品skc-id
        url1 = base_url + newin_url[row_num] + url_end1 + str(page_num + 1) + url_end3
        page1 = requests.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        goods_infos = source1.find_all('div', class_ = 'chip')
        for goods_info in goods_infos:
            skc_id = goods_info.find('a').get('data-sku').replace('\n',"")
            ## 把商品信息写入到 csv 文件中
            csv_writer.writerow([skc_id, newin_date[row_num]])
        ## 显示进度
        print(newin_date[row_num], '第', page_num+1, '页')
        print('############')
        # 每翻一页，休息一下，5秒钟 
        time.sleep(5)
## 关闭csv文件
csv_file.close()

2018-11-22 第 1 页
############
2018-11-22 第 2 页
############
2018-11-10 第 1 页
############
2018-11-10 第 2 页
############
2018-11-10 第 3 页
############
2018-10-25 第 1 页
############
2018-10-25 第 2 页
############
2018-10-25 第 3 页
############
2018-10-25 第 4 页
############
2018-10-25 第 5 页
############
2018-10-11 第 1 页
############
2018-10-11 第 2 页
############
2018-10-11 第 3 页
############
2018-10-11 第 4 页
############


In [ ]:
###################

# 以上是主要的脚本，如有需要，可用下面的脚本调试

###################

In [215]:
###################
# 以下是单独爬取的脚本
###################

url1 = base_url + cat_list.cat2_href[1] + url_end1 + str(1) + url_end2
page1 = requests.get(url1)
source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
# 商品信息
goods_infos = source1.find_all('div', class_ = 'chip')
for goods_info in goods_infos:
    goods_name = goods_info.find('span', class_ = 'chip_name').string.replace('\n',"")
    skc_id = goods_info.find('a').get('data-sku').replace('\n',"")
    sale_price = goods_info.find('p', class_ = 'chip_price').span.string.replace('\n',"")
    try:
        origin_price = goods_info.find('p', class_ = 'chip_price').find('del').string
    except Exception as e:
        origin_price = sale_price
    href = goods_info.find('a').get('href')
    # 商品详情页面中的商品辅信息和图片
    ## 页面
    url2 = base_url + href
    page2 = requests.get(url2)
    source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
    ## 商品信息
    goods_info2 = source2.find('dl', class_ = 'pro_description').find_all('dl')
    style = goods_info2[1].dd.string.replace('\n',"")
    material = goods_info2[3].dd.string.replace('\n',"")
    length = goods_info2[4].dd.string.replace('\n',"")
    stereotype = goods_info2[5].dd.string.replace('\n',"")
    img_src = source2.find('div', class_ = 'product_img_show').find('img').get('src')
    print(goods_name)
    print(skc_id)
    print(sale_price)
    print(origin_price)
    print(href)
    print(style)
    print(material)
    print(length)
    print(stereotype)
    print('----')

棉质宽松印花圆领T恤
3GC3025810090
￥499
None
/p/3GC3025810090.shtml
模仿粉笔黑白涂鸦风格，Colourful字母印花以贴布绣形式不对称饰于前幅，个性中展现年轻活力气息；圆领短袖轮廓，配合宽松版型设计，呼应休闲随性基调；甄选棉质面料打造，亲肤柔软透气，穿着舒适自在；时髦休闲单品，下装配入短裤或束脚哈伦裤，加入低帮鞋，轻松打造阳光帅气潮男形象
面料:棉96.0% 氨纶4.0%印花贴布:棉100%
适中
宽松
----
纯棉纯色中长宽松T恤
3GC3025880000
￥399
None
/p/3GC3025880000.shtml
静默淡然的纯色衣身，传递出宁静质朴的现代生活审美，勾勒雅致素净质感；双肩同色贴布绣装饰，赋予衣服更多时尚看点，低调中展现个性；中长款长度剪裁，配合流畅线条宽松版型，构筑悠闲舒适廓形；甄选亲肤质感纯棉面料，柔软透气吸汗，穿着舒适自在；时髦休闲款式，下装配入短裤或束脚哈伦裤，加入低帮鞋，轻松打造阳光帅气潮男形象
面料:棉100%撞料:锦纶100%
中长
宽松
----
纯棉纯色中长宽松T恤
3GC3025880090
￥399
None
/p/3GC3025880090.shtml
静默淡然的纯色衣身，传递出宁静质朴的现代生活审美，勾勒雅致素净质感；双肩同色贴布绣装饰，赋予衣服更多时尚看点，低调中展现个性；中长款长度剪裁，配合流畅线条宽松版型，构筑悠闲舒适廓形；甄选亲肤质感纯棉面料，柔软透气吸汗，穿着舒适自在；时髦休闲款式，下装配入短裤或束脚哈伦裤，加入低帮鞋，轻松打造阳光帅气潮男形象
面料:棉100%撞料:锦纶100%
中长
宽松
----
纯棉刺绣圆领短袖T恤
3GC3026400000
￥499
None
/p/3GC3026400000.shtml
运用精巧细致的刺绣工艺，勾画乖张眼睛图案，透着宗教神秘色彩，搭配英文Slogan印花与花蛇图案，亮点出挑个性吸睛；圆领短袖轮廓，线条处理利落称身，呼应休闲基调；甄选亲肤质感纯棉面料，柔软透气吸汗，穿着舒适自在；时髦休闲款式，下装配入短裤或束脚哈伦裤，加入低帮鞋，轻松打造阳光帅气潮男形象
面料:棉100%里布:聚酯纤维100%
适中
合体
----
纯棉刺绣圆领短袖T恤
3GC3026400090
￥499
None
/p/3GC302640009

In [154]:
## 保存商品主图到本地文件夹
img = requests.get(img_src, timeout = 5)
img_name = "trendiano_goods_img/"+ skc_id + ".jpg"
fp = open(img_name, 'wb')
fp.write(img.content)
fp.close()

http://img1.ochirly.com.cn/wcsstore/TrendyCatalogAssetStore/images/trendy/trendiano/2018/b/3GC2025920000/3GC2025920000_m_1.jpg
trendiano_goods_img/3GC2025920000.jpg


In [9]:
## 单个商品的主信息
url1 = 'http://www.trendiano.com/T_shirts/Long_sleeved_shirts/list-2-30-listedDate%20desc-0-0-0-0-0.shtml'
page1 = requests.get(url1, headers = header)
source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
goods_info = source1.find('div', class_ = 'chip')
goods_name = goods_info.find('span', class_ = 'chip_name').string.replace('\n',"")
skc_id = goods_info.find('a').get('data-sku').replace('\n',"")
sale_price = goods_info.find('p', class_ = 'chip_price').span.string.replace('\n',"")
try:
    origin_price = goods_info.find('p', class_ = 'chip_price').find('del').string
except Exception as e:
    origin_price = None
href = goods_info.find('a').get('href')
print(goods_name)
print(skc_id)
print(sale_price)
print(origin_price)
print(href)

宽松纯棉格子长袖衬衫
3GI1015630955
￥489
￥699
/p/3GI1015630955.shtml


In [10]:
# 商品详情页面中的商品辅信息和图片
## 页面
url2 = 'http://www.trendiano.com/p/3GI1015630955.shtml'
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
## 商品辅信息
goods_info2 = source2.find('dl', class_ = 'pro_description').find_all('dl')
style = goods_info2[1].dd.string.replace('\n',"")
material = goods_info2[3].dd.string.replace('\n',"")
length = goods_info2[4].dd.string.replace('\n',"")
stereotype = goods_info2[5].dd.string.replace('\n',"")
img_src = source2.find('div', class_ = 'product_img_show').find('img').get('src')
print(goods_name)
print(skc_id)
print(sale_price)
print(origin_price)
print(href)
print(style)
print(material)
print(length)
print(stereotype)
print(img_src)

宽松纯棉格子长袖衬衫
3GI1015630955
￥489
￥699
/p/3GI1015630955.shtml
细格子元素的加入，赋予衣衫清新干净气质，尤显帅气文雅；个性化slogan成功吸睛抢眼，以精致刺绣手法呈现，更显时尚别致；弧形衣摆剪裁，添注轻松休闲味道，一改正式通勤传统；宽松版型，巧妙修饰及包容身材，舒适度提升；精选纯棉面料打造，穿着舒适自在透气；可单穿可作为外套亦可内搭，可塑性强，值得入手；混搭套头衫，下装配以合体长裤、板鞋，并加入斜挎小包，通身造型干净青春，释放时髦活力之感
面料:棉100%(绣花线除外)
长款
宽松
http://img1.ochirly.com.cn/wcsstore/TrendyCatalogAssetStore/images/trendy/trendiano/2018/a/3GI1015630955/3GI1015630955_m_1.jpg


In [42]:
## 所有 URL
cat1_url = source.find('div', class_ = "nav2013_ul_box").ul.find_all('li')
for item in cat1_url:
    cat1_href = item.find_all('a')
    for a in cat1_href:
        print(a.get('href'))

/index.shtml
/a/news/active/list.shtml
/a/news/new_in/list.shtml
/sale/list.shtml
/sale/T_shirts/list.shtml
/sale/Shirts/list.shtml
/sale/Pants/list.shtml
/sale/Hoodies/list.shtml
/sale/Coats/list.shtml
/sale/Accessories/list.shtml
/New_in/list.shtml
/New_in/New_in_8_16/list.shtml
/New_in/New_in_8_2/list.shtml
/New_in/New_in_7_19/list.shtml
/New_in/New_in_7_5/list.shtml
/New_in/New_in_2018SS/list.shtml
/New_in/All_new_products/list.shtml
/THREE_MAJOR_LABELS/list.shtml
/THREE_MAJOR_LABELS/Golden_Labels/list.shtml
/THREE_MAJOR_LABELS/Black_Labels/list.shtml
/THREE_MAJOR_LABELS/White_Labels/list.shtml
/T_shirts/list.shtml
/T_shirts/Sleeveless_T_shirts/list.shtml
/T_shirts/Short_sleeved_T_shirts/list.shtml
/T_shirts/Long_sleeved_T_shirts/list.shtml
/T_shirts/POLO_shirts/list.shtml
/Shirts/list.shtml
/Shirts/Long_sleeved_shirts/list.shtml
/Shirts/Short_sleeved_shirts/list.shtml
/Shirts/Sleeveless_shirts/list.shtml
/Pants/list.shtml
/Pants/Casual_pants/list.shtml
/Pants/Jeans/list.shtml
/Coa

In [81]:
# 获取一级类目的名称，strip()函数去除字符串的空格
for item in source.find('div', class_ = "nav2013_ul_box").ul.find_all('li'):
    cat_href = item.find_all('a')
    for a_str in cat_href:
        print(a_str.string.strip())

首页
品牌动向
新品资讯
春夏大减价
T恤/POLO衫
衬衫
裤装
卫衣
外套
配饰
全部新品
8月16号到店
8月2号到店
7月19号到店
7月5号到店
2018春夏合集
全部新品
三大系列
金标系列
黑标系列
白标系列
T恤/POLO衫
无袖T恤
短袖T恤
长袖T恤
POLO
衬衫
长袖衬衫
短袖衬衫
无袖衬衫
裤装
休闲裤
牛仔裤
薄外套
卫衣
毛织
风衣
夹克
西服
鞋
凉鞋/拖鞋
休闲鞋
配饰
帽
项链
包
袜
